# Workshop 1: The File Organizer Agent

### Build an AI assistant that organizes a messy folder — automatically

---

**How this works:** All the code is pre-written. You just run each cell (press **Shift+Enter**) and observe what happens. The goal is to understand *how* AI agents work, not to write code.

By the end, you'll understand the 3 building blocks of every AI agent:
1. **Tools** — what the AI *can* do
2. **System Prompt** — how the AI *thinks*
3. **Agent Loop** — the engine that keeps it going until the job is done

---

## Section 1: What's in the Messy Folder?

Imagine your intern dumped 12 random files on your desktop. Photos, spreadsheets, code files, documents — all mixed up in one folder. No organization whatsoever.

Let's build an AI assistant that organizes them **automatically**.

First, let's see the mess:

In [ ]:
# Just run this cell! Press Shift+Enter
import os

messy_path = os.path.join(os.getcwd(), "messy_folder")
files = sorted(os.listdir(messy_path))

print(f"The messy folder has {len(files)} files:\n")
for f in files:
    if f.startswith(".") or f == "__pycache__":
        continue
    ext = os.path.splitext(f)[1] or "(no extension)"
    print(f"   {f:<30} {ext}")

Look at that mess! Photos (`.jpg`, `.png`), spreadsheets (`.csv`, `.xlsx`), code (`.py`, `.sql`), documents (`.pdf`, `.txt`, `.md`) — all jumbled together.

A human would sort these by type. **Can AI do the same?** Let's find out.

---

## Section 2: The Toolbox — What Can Our AI Do?

Think about hiring a new assistant at your company. On their first day, you'd say:

> *"Here's what you can do: you can **look at** files on the shared drive, you can **create** new folders, and you can **move** files between folders."*

That's **exactly** what we do with AI. We define a set of **tools** — and describe each one in plain English so the AI understands what it can do.

Let's look at our 3 tools:

In [ ]:
# Just run this cell! Press Shift+Enter
# These are the tool DEFINITIONS — they tell the AI what tools exist

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "List all files and folders in a directory. Returns a string with one entry per line.",
            "parameters": {
                "type": "object",
                "properties": {
                    "folder_path": {
                        "type": "string",
                        "description": "The path to the directory to list",
                    }
                },
                "required": ["folder_path"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "create_folder",
            "description": "Create a new folder at the given path. Creates parent folders if needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "folder_path": {
                        "type": "string",
                        "description": "The path of the folder to create",
                    }
                },
                "required": ["folder_path"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "move_file",
            "description": "Move a file from one location to another.",
            "parameters": {
                "type": "object",
                "properties": {
                    "source_path": {
                        "type": "string",
                        "description": "The current path of the file",
                    },
                    "destination_path": {
                        "type": "string",
                        "description": "The new path for the file",
                    },
                },
                "required": ["source_path", "destination_path"],
            },
        },
    },
]

print("3 tools defined:")
for tool in TOOLS:
    name = tool["function"]["name"]
    desc = tool["function"]["description"]
    params = list(tool["function"]["parameters"]["properties"].keys())
    print(f"\n   {name}")
    print(f"   Description: {desc}")
    print(f"   Parameters:  {', '.join(params)}")

### Breaking down the tool definitions

Each tool has 3 parts:

| Part | What it does | Example |
|------|-------------|--------|
| **name** | A short ID the AI uses to call the tool | `"list_files"` |
| **description** | Plain English explanation of what it does | `"List all files and folders in a directory"` |
| **parameters** | What inputs the tool needs | `folder_path` (a string) |

Notice: **we describe each tool in English.** The AI reads these descriptions to understand what each tool does. This is like writing a job description for a new hire.

> **Key PM Insight:** The tool descriptions are YOUR product requirements. Bad description = confused AI = bad product. If you wrote *"does stuff with files"* instead of *"List all files and folders in a directory"*, the AI wouldn't know when to use this tool.

---

## Section 3: The Tool Implementations — The Actual Work

The tool *definitions* above told the AI what's available. Now we need the actual code that **does the work**.

Think of it this way:
- The AI decides **WHAT** to do (*"I need to move photo.jpg into the images folder"*)
- These functions do **HOW** (*actually moves the file on disk*)

The AI is the brain. These functions are the hands.

In [ ]:
# Just run this cell! Press Shift+Enter
import shutil

def list_files(folder_path):
    """List all files and folders in a directory."""
    entries = os.listdir(folder_path)
    entries.sort()
    lines = []
    for entry in entries:
        if entry.startswith(".") or entry == "__pycache__":
            continue
        full_path = os.path.join(folder_path, entry)
        if os.path.isdir(full_path):
            lines.append(f"[DIR]  {entry}")
        else:
            lines.append(f"[FILE] {entry}")
    return "\n".join(lines) if lines else "(empty directory)"


def create_folder(folder_path):
    """Create a new folder (and any parent folders)."""
    os.makedirs(folder_path, exist_ok=True)
    return f"Folder created: {folder_path}"


def move_file(source_path, destination_path):
    """Move a file from source to destination."""
    shutil.move(source_path, destination_path)
    return f"Moved: {source_path} -> {destination_path}"


def execute_tool(tool_name, tool_args):
    """Dispatch a tool call to the right function."""
    if tool_name == "list_files":
        return list_files(tool_args["folder_path"])
    elif tool_name == "create_folder":
        return create_folder(tool_args["folder_path"])
    elif tool_name == "move_file":
        return move_file(tool_args["source_path"], tool_args["destination_path"])
    else:
        return f"Unknown tool: {tool_name}"

print("Tool implementations loaded!")
print("")
print("Quick test — listing our messy folder:")
print(list_files(messy_path))

As a PM, you don't need to write these functions. Your engineers do. But you need to understand the contract:

- **The AI picks the tool** (based on the definitions from Section 2)
- **The code does the work** (these functions above)
- **The result goes back to the AI** (so it knows what happened)

This separation is powerful: you can change what the tools *do* without changing how the AI *thinks*, and vice versa.

---

## Section 4: The Brain — System Prompt

Before the AI starts working, we give it instructions. This is called the **system prompt** — think of it as the product brief for your AI assistant.

It answers: *Who are you? What's your job? How should you approach it?*

In [ ]:
# Just run this cell! Press Shift+Enter

SYSTEM_PROMPT = """You are a file organizer agent. Your job is to organize files in a
messy folder into tidy subfolders based on file type.

Use these categories:
- images/    -> .jpg, .jpeg, .png, .gif, .svg
- documents/ -> .pdf, .doc, .docx, .txt, .md
- data/      -> .csv, .xlsx, .json, .sql, .xml
- code/      -> .py, .js, .ts, .html, .css

Steps:
1. First, list the files in the folder to see what needs organizing.
2. Create the necessary subfolders.
3. Move each file into the correct subfolder.
4. List the folder again to confirm everything is organized.

Work autonomously — do not ask the user for input. Just organize the files."""

print("System Prompt:")
print("=" * 60)
print(SYSTEM_PROMPT)
print("=" * 60)

**Read it carefully.** We told the AI:

1. **Who you are:** *"You are a file organizer agent"*
2. **The categories:** images, documents, data, code — with specific file extensions
3. **The strategy:** list first, create folders, move files, verify
4. **The behavior:** *"Work autonomously — do not ask the user"*

This is YOUR **product thinking** translated into AI instructions. The categories? That's a product decision. The 4-step strategy? That's a workflow you designed.

> **Think about it:** What would happen if we removed the categories from the prompt? Would the AI still know where to put files? Would it invent its own categories? We'll experiment with this later...

---

## Section 5: The Agent Loop — Watch It Work!

Now let's put it all together and watch the AI organize files **autonomously**.

This is the **agent loop** — the engine at the heart of every AI agent:

```
Ask AI  -->  AI picks a tool  -->  Run the tool  -->  Tell AI the result  -->  Repeat
                                                                                  |
                                                         (until AI stops picking tools)
```

First, let's set up the AI connection and create a safe copy of our messy folder so we can experiment without losing the original files:

In [ ]:
# Just run this cell! Press Shift+Enter
# This sets up the AI connection and creates a demo copy of the messy folder

import json
from openai import OpenAI

# --- API Setup (auto-detects your provider) ---
API_KEY = (
    os.getenv("API_KEY")
    or os.getenv("OPENAI_API_KEY")
    or os.getenv("ANTHROPIC_API_KEY")
    or os.getenv("GEMINI_API_KEY")
)

if API_KEY and API_KEY.startswith("sk-ant-"):
    BASE_URL = "https://api.anthropic.com/v1/"
    MODEL = "claude-sonnet-4-20250514"
elif API_KEY and API_KEY.startswith("sk-"):
    BASE_URL = "https://api.openai.com/v1"
    MODEL = "gpt-4o"
else:
    BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
    MODEL = "gemini-2.0-flash"

BASE_URL = os.getenv("OPENAI_BASE_URL", BASE_URL)
MODEL = os.getenv("MODEL", MODEL)
MAX_TOKENS = 4096
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

print(f"AI Model: {MODEL}")
print(f"API connected!")

# --- Create a demo copy so we don't destroy the original ---
demo_path = os.path.join(os.getcwd(), "messy_folder_demo")
if os.path.exists(demo_path):
    shutil.rmtree(demo_path)
shutil.copytree(messy_path, demo_path, ignore=shutil.ignore_patterns('__pycache__', '.*'))

print(f"")
print(f"Created demo copy: messy_folder_demo/")
print(f"Files: {len([f for f in os.listdir(demo_path) if not f.startswith('.')])}")
print(f"")
print(f"Ready to run the agent!")

### Now the big moment. Run the cell below and watch the AI work!

Pay attention to the output. You'll see:
- Each **step** the AI takes
- Which **tool** it picks and why
- The **result** of each tool call
- The AI's **final summary** when it's done

In [ ]:
# Just run this cell! Press Shift+Enter
# Watch the AI organize files autonomously!

# Start the conversation
messages = [{"role": "system", "content": SYSTEM_PROMPT}]
messages.append({"role": "user", "content": f"Organize the files in the {demo_path} directory."})

print("=" * 60)
print("  FILE ORGANIZER AGENT")
print("="* 60)
print(f"  Task: Organize {demo_path}")
print("=" * 60)
print()

step = 0
while True:
    step += 1
    print(f"--- Step {step}: AI is thinking... ---")
    print()

    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        tools=TOOLS,
        messages=messages,
    )

    assistant_message = response.choices[0].message
    messages.append(assistant_message)

    # If the AI wants to use tools
    if assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            # Show what the AI decided to do
            args_display = ", ".join(f"{k}={v!r}" for k, v in tool_args.items())
            print(f"  AI wants to use tool: {tool_name}")
            print(f"  Arguments: {args_display}")
            print()

            # Run the tool
            result = execute_tool(tool_name, tool_args)

            # Show the result
            print(f"  Tool result:")
            for line in str(result).split("\n"):
                print(f"    {line}")
            print()

            # Tell the AI what happened
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            })
    else:
        # No more tool calls — the AI is done!
        print("=" * 60)
        print("  AGENT FINISHED!")
        print("=" * 60)
        if assistant_message.content:
            print()
            print("  AI's final message:")
            print()
            for line in assistant_message.content.split("\n"):
                print(f"    {line}")
        print()
        break

print(f"Total steps: {step}")

### Did you see that?

The AI:
1. **Looked** at the files (used `list_files`)
2. **Figured out** the categories on its own
3. **Created** the right folders (used `create_folder`)
4. **Moved** every file to the right place (used `move_file`)
5. **Verified** the result (used `list_files` again)

You never told it *"put .jpg files in images/"*. The AI **figured that out** from the system prompt and the file names.

Let's verify the organized folder:

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's see the organized result

print("Organized folder structure:")
print()
for item in sorted(os.listdir(demo_path)):
    if item.startswith(".") or item == "__pycache__":
        continue
    item_path = os.path.join(demo_path, item)
    if os.path.isdir(item_path):
        sub_files = [f for f in os.listdir(item_path) if not f.startswith(".")]
        print(f"  {item}/")
        for f in sorted(sub_files):
            print(f"    {f}")
    else:
        print(f"  {item}  (not moved!)")

---

## Section 6: The Aha Moment — What Just Happened?

Let's recap what you just saw:

| You provided | The AI did |
|---|---|
| **3 tools** (list, create folder, move) | Decided **when** to use each tool |
| **A system prompt** (the strategy) | Decided **which** folders to create |
| **An agent loop** (the engine) | Decided **which** file goes where |

You gave the AI **capabilities** and **instructions**. The AI made **all the decisions**.

### This is how EVERY AI agent works:

```
Tools  +  Prompt  +  Loop  =  Agent
```

- **Tools** = what actions are available
- **Prompt** = what strategy to follow
- **Loop** = keep going until the job is done

Whether it's a coding assistant, a customer support bot, or a data analyst — they all follow this exact pattern. The only differences are the tools, the prompt, and the termination condition.

---

## Section 7: Experiment Corner

The best way to understand agents is to **break things** and see what happens. Let's run some experiments!

### Experiment 1: What if we REMOVE the move_file tool?

The AI can see files and create folders, but it **can't move anything**. What will it do?

In [ ]:
# Just run this cell! Press Shift+Enter
# First, reset the demo folder

if os.path.exists(demo_path):
    shutil.rmtree(demo_path)
shutil.copytree(messy_path, demo_path, ignore=shutil.ignore_patterns('__pycache__', '.*'))
print("Demo folder reset!")
print(f"Files: {len([f for f in os.listdir(demo_path) if not f.startswith('.')])}")

In [ ]:
# Just run this cell! Press Shift+Enter
# Run the agent WITHOUT the move_file tool

# Only 2 tools: list_files and create_folder (no move_file!)
LIMITED_TOOLS = [t for t in TOOLS if t["function"]["name"] != "move_file"]
print(f"Tools available: {[t['function']['name'] for t in LIMITED_TOOLS]}")
print(f"(Notice: move_file is MISSING!)")
print()

messages = [{"role": "system", "content": SYSTEM_PROMPT}]
messages.append({"role": "user", "content": f"Organize the files in the {demo_path} directory."})

print("=" * 60)
print("  EXPERIMENT 1: No move_file tool")
print("=" * 60)
print()

step = 0
while step < 15:  # Safety limit
    step += 1
    print(f"--- Step {step} ---")

    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        tools=LIMITED_TOOLS,
        messages=messages,
    )

    assistant_message = response.choices[0].message
    messages.append(assistant_message)

    if assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            args_display = ", ".join(f"{k}={v!r}" for k, v in tool_args.items())
            print(f"  Tool: {tool_name}({args_display})")

            result = execute_tool(tool_name, tool_args)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            })
    else:
        print()
        print("=" * 60)
        print("  AI's response (no more tools to call):")
        print("=" * 60)
        if assistant_message.content:
            print()
            for line in assistant_message.content.split("\n"):
                print(f"    {line}")
        print()
        break

print(f"Steps taken: {step}")

**What happened?** The AI could see the mess and create folders, but it **couldn't actually move files**. It likely told you it couldn't complete the task or tried to work around the limitation.

> **PM Insight:** Removing a tool is like removing a feature from your product. The AI adapts, but the user experience suffers. Your job as PM is to make sure the AI has the right tools for the job.

---

### Experiment 2: What if we change the categories?

Instead of organizing by file type (images, documents, data, code), let's ask the AI to sort by **"work" vs "personal"**. Same tools, different strategy.

In [ ]:
# Just run this cell! Press Shift+Enter
# Reset the demo folder first

if os.path.exists(demo_path):
    shutil.rmtree(demo_path)
shutil.copytree(messy_path, demo_path, ignore=shutil.ignore_patterns('__pycache__', '.*'))
print("Demo folder reset!")

In [ ]:
# Just run this cell! Press Shift+Enter
# New prompt: organize by "work" vs "personal" instead of file type

CUSTOM_PROMPT = """You are a file organizer agent. Your job is to organize files in a
messy folder based on whether they seem WORK-RELATED or PERSONAL.

Use these categories:
- work/     -> files that seem work-related (reports, budgets, code, databases, etc.)
- personal/ -> files that seem personal (vacation photos, shopping lists, etc.)

Steps:
1. First, list the files in the folder to see what needs organizing.
2. Create the necessary subfolders.
3. Look at each filename and decide if it's work or personal.
4. Move each file into the correct subfolder.
5. List the folder again to confirm everything is organized.

Work autonomously — do not ask the user for input. Just organize the files."""

messages = [{"role": "system", "content": CUSTOM_PROMPT}]
messages.append({"role": "user", "content": f"Organize the files in the {demo_path} directory."})

print("=" * 60)
print("  EXPERIMENT 2: Work vs Personal categories")
print("=" * 60)
print()

step = 0
while True:
    step += 1
    print(f"--- Step {step} ---")

    response = client.chat.completions.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        tools=TOOLS,
        messages=messages,
    )

    assistant_message = response.choices[0].message
    messages.append(assistant_message)

    if assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            args_display = ", ".join(f"{k}={v!r}" for k, v in tool_args.items())
            print(f"  Tool: {tool_name}({args_display})")

            result = execute_tool(tool_name, tool_args)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result),
            })
    else:
        print()
        print("=" * 60)
        print("  DONE!")
        print("=" * 60)
        if assistant_message.content:
            print()
            for line in assistant_message.content.split("\n"):
                print(f"    {line}")
        print()
        break

print(f"Steps taken: {step}")

In [ ]:
# Just run this cell! Press Shift+Enter
# Let's see how the AI categorized the files

print("How the AI organized by work vs personal:")
print()
for item in sorted(os.listdir(demo_path)):
    if item.startswith(".") or item == "__pycache__":
        continue
    item_path = os.path.join(demo_path, item)
    if os.path.isdir(item_path):
        sub_files = [f for f in os.listdir(item_path) if not f.startswith(".")]
        print(f"  {item}/")
        for f in sorted(sub_files):
            print(f"    {f}")
    else:
        print(f"  {item}  (not moved)")

**Interesting!** Same tools, same files, but a **completely different result** — just because we changed the prompt.

Did the AI put `vacation_photo.jpg` in personal? What about `team_photo.jpg` — is that work or personal? These are judgment calls, and the AI made them based on file names alone.

> **PM Insight:** The system prompt is your **product strategy**. Same tools + different prompt = different product. This is why prompt engineering matters so much for AI products.

---

## Section 8: PM Takeaways

Here's what you should take away from this workshop:

### The Agent Formula

```
Tools  +  Prompt  +  Loop  =  Agent
```

### What each piece means for PMs

| Component | What it is | Your PM role |
|-----------|-----------|-------------|
| **Tools** | The actions the AI can take | Define **product capabilities** — what should your AI be able to do? |
| **System Prompt** | The strategy and rules | Define **product strategy** — how should your AI think and behave? |
| **Agent Loop** | The engine that keeps it going | Define **termination conditions** — when is the job done? |
| **Tool Descriptions** | How tools are explained to the AI | Write **clear requirements** — bad descriptions = confused AI |

### Key insights

- **Tools are product features.** Removing `move_file` was like shipping without a core feature — the AI could plan but not execute.

- **The system prompt is your product strategy.** Same tools + different prompt = completely different product behavior (file type vs work/personal).

- **The AI makes decisions, not you.** You defined categories, but the AI decided which file goes where. That's the power — and the risk — of agents.

- **The loop needs a stop condition.** Without knowing when to stop, agents run forever (and burn your API budget). Termination design is critical.

- **As a PM, your job is to design the right tools and write the right prompts.** The engineering is the loop. The product thinking is everything else.

In [ ]:
# Just run this cell! Press Shift+Enter
# Clean up the demo folder

if os.path.exists(demo_path):
    shutil.rmtree(demo_path)
    print("Demo folder cleaned up.")
print("")
print("Workshop complete! The original messy_folder/ is untouched.")